In [ ]:
import pandas as pd
import os
import re

# Base directory - adjust as needed
base_dir = 'Encoder-BS32-SL64/npu'

# Store results for each folder
folder_results = []
grand_total = 0

print("="*80)
print("NPU Folder Analysis")
print("="*80)
print()

# Process folders 2 through 26
for folder_num in range(2, 10):
    folder_name = str(folder_num)
    txt_file = os.path.join(base_dir, folder_name, 'SA_stage_E.txt')
    
    if not os.path.exists(txt_file):
        print(f"Folder {folder_num}: File not found, skipping...")
        continue
    
    # Read the text file
    with open(txt_file, 'r') as f:
        content = f.read()
    
    # Parse the table - skip header lines and extract data
    lines = content.split('\n')
    
    # Find where the actual data starts (after the === line)
    data_started = False
    param_load_sum = 0
    expert_63_total_compute = 0
    
    for line in lines:
        # Skip empty lines and header lines
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line:
            continue
        
        # Split by whitespace
        parts = line.split()
        
        if len(parts) >= 6:  # Should have: Expert Number, param_load, fc1, gelu, fc2, total compute
            try:
                expert_num = int(parts[0])
                param_load = int(parts[1])
                total_compute = int(parts[5])
                
                # Sum all param_load values
                param_load_sum += param_load
                
                # Store expert 63's total compute
                if expert_num == 63:
                    expert_63_total_compute = total_compute
            except (ValueError, IndexError):
                continue
    
    # Calculate folder total
    folder_total = param_load_sum + expert_63_total_compute
    
    # Store result
    folder_results.append({
        'Folder': folder_num,
        'Sum of param_load': param_load_sum,
        'Expert 63 total compute': expert_63_total_compute,
        'Folder Total': folder_total
    })
    
    # Add to grand total
    grand_total += folder_total
    
    # Print result
    print(f"Folder {folder_num}:")
    print(f"  Sum of all param_load: {param_load_sum:,}")
    print(f"  Expert 63 total compute: {expert_63_total_compute:,}")
    print(f"  Folder Total: {folder_total:,}")
    print()

# Create summary DataFrame
summary_df = pd.DataFrame(folder_results)

print("="*80)
print("Summary Table")
print("="*80)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print(f"GRAND TOTAL (Sum of all folders): {grand_total:,}")
print("="*80)

# Display DataFrame
summary_df


NPU Folder Analysis

Folder 2:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 143,777
  Folder Total: 90,650,785

Folder 3:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 169,386
  Folder Total: 90,676,394

Folder 4:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 172,874
  Folder Total: 90,679,882

Folder 5:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 154,641
  Folder Total: 90,661,649

Folder 6:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 154,636
  Folder Total: 90,661,644

Folder 7:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 40,176
  Folder Total: 90,547,184

Folder 8:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 150,366
  Folder Total: 90,657,374

Folder 9:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 137,950
  Folder Total: 90,644,958

Folder 10:
  Sum of all param_load: 90,507,008
  Expert 63 total compute: 143,777
  Folder Total: 90,650,785

,Folder,Sum of param_load,Expert 63 total compute,Folder Total
0,2,90507008,143777,90650785
1,3,90507008,169386,90676394
2,4,90507008,172874,90679882
3,5,90507008,154641,90661649
4,6,90507008,154636,90661644
5,7,90507008,40176,90547184
6,8,90507008,150366,90657374
7,9,90507008,137950,90644958
8,10,90507008,143777,90650785
9,11,90507008,169386,90676394


In [ ]:
import pandas as pd
import os
import re

# Base directory - adjust as needed
base_dir = 'Encoder-BS32-SL64/pim'

# Store results for each folder
folder_results = []
grand_total = 0

print("="*80)
print("PIM Folder Analysis")
print("="*80)
print()

# Process folders 2 through 26
for folder_num in range(2, 10):
    folder_name = str(folder_num)
    txt_file = os.path.join(base_dir, folder_name, 'SA_stage_E.txt')
    
    if not os.path.exists(txt_file):
        print(f"Folder {folder_num}: File not found, skipping...")
        continue
    
    # Read the text file
    with open(txt_file, 'r') as f:
        content = f.read()
    
    # Parse activation movements
    activation_1 = 0
    activation_2 = 0
    
    # Extract activation movements from the file
    for line in content.split('\n'):
        if 'activation_movement_1' in line and 'cycles' in line:
            # Extract number before 'cycles'
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_1 = int(match.group(1))
        elif 'activation_movement_2' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_2 = int(match.group(1))
    
    # Parse the expert table
    lines = content.split('\n')
    total_compute_sum = 0
    
    for line in lines:
        # Skip empty lines and header lines
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line or 'Note:' in line or 'Activation' in line or '-' in line[:3]:
            continue
        
        # Split by whitespace
        parts = line.split()
        
        if len(parts) >= 5:  # Should have: Expert Number, fc1, gelu, fc2, total compute
            try:
                expert_num = int(parts[0])
                total_compute = int(parts[4])  # Last column is total compute
                
                # Sum all total compute values
                total_compute_sum += total_compute
            except (ValueError, IndexError):
                continue
    
    # Calculate folder total
    folder_total = total_compute_sum + activation_1 + activation_2
    
    # Store result
    folder_results.append({
        'Folder': folder_num,
        'Sum of total compute': total_compute_sum,
        'Activation 1': activation_1,
        'Activation 2': activation_2,
        'Folder Total': folder_total
    })
    
    # Add to grand total
    grand_total += folder_total
    
    # Print result
    print(f"Folder {folder_num}:")
    print(f"  Sum of all total compute: {total_compute_sum:,}")
    print(f"  Activation movement 1: {activation_1:,}")
    print(f"  Activation movement 2: {activation_2:,}")
    print(f"  Folder Total: {folder_total:,}")
    print()

# Create summary DataFrame
summary_df = pd.DataFrame(folder_results)

print("="*80)
print("Summary Table")
print("="*80)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print(f"GRAND TOTAL (Sum of all folders): {grand_total:,}")
print("="*80)

# Display DataFrame
summary_df


PIM Folder Analysis

Folder 2:
  Sum of all total compute: 605,457,664
  Activation movement 1: 2,301,072
  Activation movement 2: 2,290,384
  Folder Total: 610,049,120

Folder 3:
  Sum of all total compute: 605,530,880
  Activation movement 1: 2,301,072
  Activation movement 2: 2,290,384
  Folder Total: 610,122,336

Folder 4:
  Sum of all total compute: 605,532,928
  Activation movement 1: 2,301,072
  Activation movement 2: 2,290,384
  Folder Total: 610,124,384

Folder 5:
  Sum of all total compute: 605,630,464
  Activation movement 1: 2,301,072
  Activation movement 2: 2,290,384
  Folder Total: 610,221,920

Folder 6:
  Sum of all total compute: 605,578,752
  Activation movement 1: 2,301,072
  Activation movement 2: 2,290,384
  Folder Total: 610,170,208

Folder 7:
  Sum of all total compute: 605,531,392
  Activation movement 1: 2,301,072
  Activation movement 2: 2,290,384
  Folder Total: 610,122,848

Folder 8:
  Sum of all total compute: 605,428,992
  Activation movement 1: 2,301,072


,Folder,Sum of total compute,Activation 1,Activation 2,Folder Total
0,2,605457664,2301072,2290384,610049120
1,3,605530880,2301072,2290384,610122336
2,4,605532928,2301072,2290384,610124384
3,5,605630464,2301072,2290384,610221920
4,6,605578752,2301072,2290384,610170208
5,7,605531392,2301072,2290384,610122848
6,8,605428992,2301072,2290384,610020448
7,9,605432832,2301072,2290384,610024288
8,10,605457664,2301072,2290384,610049120
9,11,605530880,2301072,2290384,610122336


In [ ]:
import pandas as pd
import os
import re

# Base directories
base_dir = 'Encoder-BS32-SL64'
npu_base = os.path.join(base_dir, 'npu')
pim_base = os.path.join(base_dir, 'pim')

# Store results for each folder
folder_results = []
grand_total = 0

print("="*80)
print("HYBRID MODE Analysis (Top 4 on NPU, Rest on PIM)")
print("="*80)
print()

# Process folders 2 through 26
for folder_num in range(2, 10):
    # Determine which layer file to use (cycling through 2-9)
    layer_num = ((folder_num - 2) % 8) + 2
    layer_file = os.path.join(base_dir, f'layer{layer_num}.txt')
    
    npu_txt = os.path.join(npu_base, str(folder_num), 'SA_stage_E.txt')
    pim_txt = os.path.join(pim_base, str(folder_num), 'SA_stage_E.txt')
    
    # Check if files exist
    if not os.path.exists(layer_file):
        print(f"Folder {folder_num}: layer{layer_num}.txt not found, skipping...")
        continue
    if not os.path.exists(npu_txt):
        print(f"Folder {folder_num}: NPU file not found, skipping...")
        continue
    if not os.path.exists(pim_txt):
        print(f"Folder {folder_num}: PIM file not found, skipping...")
        continue
    
    # Read layer file to identify top 4 experts
    with open(layer_file, 'r') as f:
        layer_content = f.read()
    
    # Parse layer file to get top 4 experts
    top_4_experts = []
    for line in layer_content.split('\n'):
        # Look for lines with expert numbers
        match = re.search(r'Expert\s+(\d+)', line)
        if match:
            expert_num = int(match.group(1))
            top_4_experts.append(expert_num)
            if len(top_4_experts) == 4:
                break
    
    if len(top_4_experts) != 4:
        print(f"Folder {folder_num}: Could not find top 4 experts in layer{layer_num}.txt, skipping...")
        continue
    
    print(f"Folder {folder_num} (using layer{layer_num}.txt):")
    print(f"  Top 4 experts: {top_4_experts}")
    
    # Read NPU file and extract data for top 4 experts
    with open(npu_txt, 'r') as f:
        npu_content = f.read()
    
    npu_param_load_sum = 0
    expert_4th_total_compute = 0
    
    for line in npu_content.split('\n'):
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line:
            continue
        
        parts = line.split()
        if len(parts) >= 6:
            try:
                expert_num = int(parts[0])
                if expert_num in top_4_experts:
                    param_load = int(parts[1])
                    total_compute = int(parts[5])
                    npu_param_load_sum += param_load
                    
                    # If this is the 4th expert (last of top 4)
                    if expert_num == top_4_experts[3]:
                        expert_4th_total_compute = total_compute
            except (ValueError, IndexError):
                continue
    
    # Read PIM file and extract data for remaining experts
    with open(pim_txt, 'r') as f:
        pim_content = f.read()
    
    # Extract activation movements
    activation_1 = 0
    activation_2 = 0
    
    for line in pim_content.split('\n'):
        if 'activation_movement_1' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_1 = int(match.group(1))
        elif 'activation_movement_2' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_2 = int(match.group(1))
    
    # Extract total compute for remaining experts (not in top 4)
    pim_total_compute_sum = 0
    
    for line in pim_content.split('\n'):
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line or 'Note:' in line or 'Activation' in line or '-' in line[:3]:
            continue
        
        parts = line.split()
        if len(parts) >= 5:
            try:
                expert_num = int(parts[0])
                # Only include experts NOT in top 4
                if expert_num not in top_4_experts:
                    total_compute = int(parts[4])
                    pim_total_compute_sum += total_compute
            except (ValueError, IndexError):
                continue
    
    # Calculate folder total (NPU and PIM run in PARALLEL, so take MAX)
    npu_time = npu_param_load_sum + expert_4th_total_compute
    pim_time = pim_total_compute_sum + activation_1 + activation_2
    folder_total = max(npu_time, pim_time)
    
    # Store result
    folder_results.append({
        'Folder': folder_num,
        'Layer File': f'layer{layer_num}.txt',
        'NPU param_load (top 4)': npu_param_load_sum,
        '4th expert total compute': expert_4th_total_compute,
        'PIM total compute (rest)': pim_total_compute_sum,
        'Activation 1': activation_1,
        'Activation 2': activation_2,
        'NPU Total': npu_time,
        'PIM Total': pim_time,
        'Folder Total': folder_total
    })
    
    # Add to grand total
    grand_total += folder_total
    
    # Print result
    print(f"  NPU Time (parallel with PIM):")
    print(f"    - Param load (top 4): {npu_param_load_sum:,}")
    print(f"    - 4th expert compute: {expert_4th_total_compute:,}")
    print(f"    - NPU Total: {npu_time:,} cycles")
    print(f"  PIM Time (parallel with NPU):")
    print(f"    - Total compute (rest 60): {pim_total_compute_sum:,}")
    print(f"    - Activation 1: {activation_1:,}")
    print(f"    - Activation 2: {activation_2:,}")
    print(f"    - PIM Total: {pim_time:,} cycles")
    print(f"  TOTAL (MAX): {folder_total:,} cycles")
    print()

# Create summary DataFrame
summary_df = pd.DataFrame(folder_results)

print("="*80)
print("Summary Table")
print("="*80)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print(f"GRAND TOTAL (Hybrid Mode - Sum of all folders): {grand_total:,}")
print("="*80)

# Display DataFrame
summary_df


HYBRID MODE Analysis (Top 4 on NPU, Rest on PIM)

Folder 2 (using layer2.txt):
  Top 4 experts: [38, 28, 21, 1]
  NPU Time (parallel with PIM):
    - Param load (top 4): 1,414,208
    - 4th expert compute: 7,562
    - NPU Total: 1,421,770 cycles
  PIM Time (parallel with NPU):
    - Total compute (rest 60): 15,656,042
    - Activation 1: 143,817
    - Activation 2: 143,149
    - PIM Total: 15,943,008 cycles
  TOTAL (MAX): 15,943,008 cycles

Folder 3 (using layer3.txt):
  Top 4 experts: [57, 60, 9, 54]
  NPU Time (parallel with PIM):
    - Param load (top 4): 1,414,144
    - 4th expert compute: 6,159
    - NPU Total: 1,420,303 cycles
  PIM Time (parallel with NPU):
    - Total compute (rest 60): 15,943,498
    - Activation 1: 143,817
    - Activation 2: 143,149
    - PIM Total: 16,230,464 cycles
  TOTAL (MAX): 16,230,464 cycles

Folder 4 (using layer4.txt):
  Top 4 experts: [15, 18, 50, 3]
  NPU Time (parallel with PIM):
    - Param load (top 4): 1,414,144
    - 4th expert compute: 5,86

,Folder,Layer File,NPU param_load (top 4),4th expert total compute,PIM total compute (rest),Activation 1,Activation 2,NPU Total,PIM Total,Folder Total
0,2,layer2.txt,1414208,7562,15656042,143817,143149,1421770,15943008,15943008
1,3,layer3.txt,1414144,6159,15943498,143817,143149,1420303,16230464,16230464
2,4,layer4.txt,1414144,5863,16056239,143817,143149,1420007,16343205,16343205
3,5,layer5.txt,1414144,6162,15972753,143817,143149,1420306,16259719,16259719
4,6,layer6.txt,1414144,6161,16007128,143817,143149,1420305,16294094,16294094
5,7,layer7.txt,1414272,6262,15977477,143817,143149,1420534,16264443,16264443
6,8,layer8.txt,1414144,6658,15944720,143817,143149,1420802,16231686,16231686
7,9,layer9.txt,1414144,6760,15866203,143817,143149,1420904,16153169,16153169


In [ ]:
import pandas as pd
import os
import re

# Base directories
base_dir = 'Encoder-BS32-SL512'
npu_base = os.path.join(base_dir, 'npu')
pim_base = os.path.join(base_dir, 'pim')

print("="*80)
print("INCREMENTAL OPTIMIZATION Analysis")
print("Finding optimal number of experts to move from PIM to NPU")
print("="*80)
print()

# Store results for each folder
all_folder_results = []
optimization_summary = []

# Process folders 2 through 26
for folder_num in range(2, 10):
    # Determine which layer file to use (cycling through 2-9)
    layer_num = ((folder_num - 2) % 8) + 2
    layer_file = os.path.join(base_dir, f'layer{layer_num}.txt')
    
    npu_txt = os.path.join(npu_base, str(folder_num), 'SA_stage_E.txt')
    pim_txt = os.path.join(pim_base, str(folder_num), 'SA_stage_E.txt')
    
    # Check if files exist
    if not os.path.exists(layer_file) or not os.path.exists(npu_txt) or not os.path.exists(pim_txt):
        print(f"Folder {folder_num}: Required files not found, skipping...")
        continue
    
    # Read layer file to get ordered list of experts (top to bottom)
    with open(layer_file, 'r') as f:
        layer_content = f.read()
    
    expert_order = []
    for line in layer_content.split('\n'):
        match = re.search(r'Expert\s+(\d+)', line)
        if match:
            expert_num = int(match.group(1))
            expert_order.append(expert_num)
    
    if len(expert_order) < 64:
        print(f"Folder {folder_num}: Could not find all experts in layer{layer_num}.txt, skipping...")
        continue
    
    # Read NPU data
    with open(npu_txt, 'r') as f:
        npu_content = f.read()
    
    npu_data = {}
    for line in npu_content.split('\n'):
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line:
            continue
        
        parts = line.split()
        if len(parts) >= 6:
            try:
                expert_num = int(parts[0])
                param_load = int(parts[1])
                total_compute = int(parts[5])
                npu_data[expert_num] = {'param_load': param_load, 'total_compute': total_compute}
            except (ValueError, IndexError):
                continue
    
    # Read PIM data
    with open(pim_txt, 'r') as f:
        pim_content = f.read()
    
    # Extract activation movements
    activation_1 = 0
    activation_2 = 0
    
    for line in pim_content.split('\n'):
        if 'activation_movement_1' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_1 = int(match.group(1))
        elif 'activation_movement_2' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_2 = int(match.group(1))
    
    pim_data = {}
    for line in pim_content.split('\n'):
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line or 'Note:' in line or 'Activation' in line or '-' in line[:3]:
            continue
        
        parts = line.split()
        if len(parts) >= 5:
            try:
                expert_num = int(parts[0])
                total_compute = int(parts[4])
                pim_data[expert_num] = {'total_compute': total_compute}
            except (ValueError, IndexError):
                continue
    
    # Now test configurations from 0 to 64 experts on NPU
    configurations = []
    
    for num_on_npu in range(0, 65):
        if num_on_npu == 0:
            # All on PIM
            npu_time = 0
            pim_compute = sum(pim_data[exp]['total_compute'] for exp in expert_order if exp in pim_data)
            pim_time = pim_compute + activation_1 + activation_2
            total_cycles = pim_time
        else:
            # Top num_on_npu experts on NPU, rest on PIM
            top_experts = expert_order[:num_on_npu]
            remaining_experts = expert_order[num_on_npu:]
            
            # NPU: param_load of top experts + total compute of last expert in top group
            npu_param_load = sum(npu_data[exp]['param_load'] for exp in top_experts if exp in npu_data)
            npu_last_compute = npu_data[top_experts[-1]]['total_compute'] if top_experts[-1] in npu_data else 0
            npu_time = npu_param_load + npu_last_compute
            
            # PIM: total compute of remaining experts
            pim_compute = sum(pim_data[exp]['total_compute'] for exp in remaining_experts if exp in pim_data)
            
            if num_on_npu == 64:
                # All on NPU, no PIM usage
                pim_time = 0
            else:
                # PIM is active
                pim_time = pim_compute + activation_1 + activation_2
            
            # Total = MAX because NPU and PIM run simultaneously
            total_cycles = max(npu_time, pim_time)
        
        configurations.append({
            'Folder': folder_num,
            'Experts on NPU': num_on_npu,
            'NPU Time': npu_time,
            'PIM Time': pim_time,
            'Total Cycles': total_cycles
        })
    
    # Find optimal configuration (minimum total cycles)
    min_config = min(configurations, key=lambda x: x['Total Cycles'])
    
    print(f"Folder {folder_num} (layer{layer_num}.txt):")
    print(f"  Optimal: {min_config['Experts on NPU']} experts on NPU")
    print(f"  NPU Time: {min_config['NPU Time']:,} | PIM Time: {min_config['PIM Time']:,}")
    print(f"  Minimum Total (MAX): {min_config['Total Cycles']:,} cycles")
    print()
    
    # Store all configurations for this folder
    all_folder_results.extend(configurations)
    
    # Store optimization summary
    optimization_summary.append({
        'Folder': folder_num,
        'Layer File': f'layer{layer_num}.txt',
        'Optimal Experts on NPU': min_config['Experts on NPU'],
        'Optimal Total Cycles': min_config['Total Cycles']
    })

# Create DataFrames
all_configs_df = pd.DataFrame(all_folder_results)
optimization_df = pd.DataFrame(optimization_summary)

print("="*80)
print("Optimization Summary - Best Configuration for Each Folder")
print("="*80)
print(optimization_df.to_string(index=False))

print("\n" + "="*80)
print(f"Sum of Optimal Totals: {optimization_df['Optimal Total Cycles'].sum():,}")
print("="*80)

# Display optimization summary
optimization_df


INCREMENTAL OPTIMIZATION Analysis
Finding optimal number of experts to move from PIM to NPU

Folder 2 (layer2.txt):
  Optimal: 55 experts on NPU
  NPU Time: 77,917,153 | PIM Time: 76,803,068
  Minimum Total (MAX): 77,917,153 cycles

Folder 3 (layer3.txt):
  Optimal: 55 experts on NPU
  NPU Time: 77,909,781 | PIM Time: 74,467,446
  Minimum Total (MAX): 77,909,781 cycles

Folder 4 (layer4.txt):
  Optimal: 55 experts on NPU
  NPU Time: 77,907,577 | PIM Time: 70,780,448
  Minimum Total (MAX): 77,907,577 cycles

Folder 5 (layer5.txt):
  Optimal: 54 experts on NPU
  NPU Time: 76,497,708 | PIM Time: 76,324,333
  Minimum Total (MAX): 76,497,708 cycles

Folder 6 (layer6.txt):
  Optimal: 55 experts on NPU
  NPU Time: 77,905,122 | PIM Time: 70,705,946
  Minimum Total (MAX): 77,905,122 cycles

Folder 7 (layer7.txt):
  Optimal: 54 experts on NPU
  NPU Time: 76,498,740 | PIM Time: 69,024,083
  Minimum Total (MAX): 76,498,740 cycles

Folder 8 (layer8.txt):
  Optimal: 53 experts on NPU
  NPU Time: 75,

,Folder,Layer File,Optimal Experts on NPU,Optimal Total Cycles
0,2,layer2.txt,55,77917153
1,3,layer3.txt,55,77909781
2,4,layer4.txt,55,77907577
3,5,layer5.txt,54,76497708
4,6,layer6.txt,55,77905122
5,7,layer7.txt,54,76498740
6,8,layer8.txt,53,75074241
7,9,layer9.txt,53,75181961
